In [1]:
from __future__ import print_function
import numpy as np
import json
import csv

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE

In [2]:
import gensim

# Sentence embedding by averaging

In [3]:
documents = []
documents_tokenized = []
with open("data.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        documents.append(row[1])
        documents_tokenized.append(gensim.utils.simple_preprocess(row[1].lower()))
print (documents[:3])

['title', 'TIL After Col. Shaw died in battle, Confederates buried him in a mass grave as an insult for leading black soldiers. Union troops tried to recover his body, but his father sent a letter saying "We would not have his body removed from where it lies surrounded by his brave and devoted soldiers."', 'TIL that in 1916 there was a proposed Amendment to the US Constitution that would put all acts of war to a national vote, and anyone voting yes would have to register as a volunteer for service in the United States Army.']


In [4]:
vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = vectorizer.fit_transform([x for x in documents])
word_to_index_tfid = vectorizer.vocabulary_

In [5]:
model = gensim.models.Word2Vec(
        documents_tokenized,
#         size=150,
#         window=100,
#         min_count=0,
    size=100, window=5, min_count=1,
        workers=10)
model.train(documents_tokenized, total_examples=len(documents_tokenized), epochs=10)

(603557, 777950)

In [6]:
len(documents)

3016

In [7]:
print(model)

Word2Vec(vocab=11903, size=100, alpha=0.025)


In [28]:
model.wv.most_similar(positive='nuclear')

[('young', 0.9996504187583923),
 ('hand', 0.9995744228363037),
 ('source', 0.9995355606079102),
 ('under', 0.9994927644729614),
 ('insisted', 0.9994838237762451),
 ('sex', 0.9994677305221558),
 ('gb', 0.9994568824768066),
 ('fast', 0.999452531337738),
 ('clear', 0.99945068359375),
 ('troops', 0.9994426369667053)]

In [16]:
def document_vector(word2vec_model, doc, doc_index):
#     # remove out-of-vocabulary words
#     doc = [word for word in doc if word in word2vec_model.wv.vocab]# and word in word_to_index_tfid]
# #     word_vectors = []
# #     for word in doc:
# #         word_vectors.append(word2vec_model[word])#*tfidf_matrix[doc_index, word_to_index_tfid[word]])
# #     print(len(word_vectors), len(word_vectors[0]))
# #     print(word_vectors[:2][:2])
#     word_vectors = word2vec_model[doc]
#     for i in range(len(word_vectors)):
#         word_vectors[i] = word_vectors[i] #* tfidf_matrix[doc_index, word_to_index_tfid[doc[i]]]
# #     if doc_index == 2:
# #         print(test[:2,:2])
# #         print(test.shape)
# #         print('\n')
# #         print(len(word_vectors), len(word_vectors[0]))
#     return np.mean(word_vectors, axis=0)
    doc = [word for word in doc if word in word2vec_model.wv.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [17]:
documents_matrix = np.zeros((len(documents_tokenized), len(model['dog'])))
for i in range(len(documents_tokenized)):
    documents_matrix[i] = document_vector(model, documents_tokenized[i], i)
print(documents_matrix.shape)

/Users/Arshi/anaconda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Users/Arshi/anaconda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(3016, 100)


In [18]:
word_to_index = {}
words_matrix = np.zeros((len(model.wv.vocab), len(model['dog'])))
i = 0
for word in model.wv.vocab:
    word_to_index[word] = i
    words_matrix[i] = model[word]
    i+=1
print(words_matrix.shape)


(11903, 100)


/Users/Arshi/anaconda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Users/Arshi/anaconda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
def closest_docs(word, k = 20):
    sims = documents_matrix.dot(words_matrix[word_to_index[word.lower()],:])
    asort = np.argsort(-sims)[:k+1]
    return [(documents[i],sims[i]/sims[asort[0]]) for i in asort[1:]]

In [25]:
closest_docs('earth')

[('Fun Fact', 1.0),
 ('Fun Fact', 1.0),
 ('Fun Fact!', 1.0),
 ('Fact day?', 0.8461115574840238),
 ('Only american', 0.841994780703958),
 ('Fun Fact: about France', 0.8014810499795191),
 ('Fun Fact: about France 2', 0.8014810499795191),
 ('Fun Fact: You blink over 100 million times in a year.', 0.7957287774780896),
 ('Fun Fact: For every human, there are 200 million insects',
  0.7822514508732252),
 ('Fun Fact: Over 200 women have run for President of the United States.',
  0.7739919176362048),
 ('Fun Fact: More than 1 out of every 5 Internet users are from China',
  0.773855914911033),
 ("Fun Fact: More collect calls are made on Father's Day than any other day of the year",
  0.7584516348351644),
 ('Fun Fact: Only one word in all of English has an X, Y, and Z in order: "Hydroxyzine."',
  0.7561798214772918),
 ("Fun Fact: You can't get less than -99 karma", 0.7554412060611668),
 ('Fun fact: 39 light years is the same as going around the world 10 billion times',
  0.7526593384708824),
 (

In [30]:
import nltk
nltk.download('punkt')


ModuleNotFoundError: No module named 'nltk'